#How to Build Machine Learning Exchange and Correlation Functionals for DFT

Made by João Paulo Almeida de Mendonça

## Basics of PYSCF

For people that have some experience with quantum chemistry nomenclature and softwares, the main obstacle in PySCF is that its structured to be "python friendly". The philosophy behind PySCF is that you define a molecule (or cell) and a method ("mean-field" of "post-mean-field") object, that combined can be used to run the SCF itself.  

Exemple - H$_2$O, 3-611G, PBE


In [1]:
!pip install --prefer-binary --upgrade pyscf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
##### A very straightforward exemple with minimal information
from pyscf import gto
from pyscf import scf

#Basic Molecule geometry and proprieties parsing. More features are explained in https://pyscf.org/user/gto.html
mol = gto.M(
    atom = '''
    O  0.000   0.000   0.000
    H  0.000  -0.757   0.587
    H  0.000   0.757   0.587 ''',
    basis = '6-311g')

#Exemple of a basic restricted Hartree--Fock calculation.
#mf.kernel returns the total energy, nothing else should be printed with mf.verbose = 0
mf = scf.RHF(mol)
mf.verbose = 0
mf.kernel()


-76.00933224032994

In [3]:
##### A similar exemple, now with few keywords added to demonstrate PySCF functionalities 
from pyscf import dft,gto,scf

#Molecule Definition
mol = gto.M(
    atom = '''
    O  0.000   0.000   0.000
    H  0.000  -0.757   0.587
    H  0.000   0.757   0.587 ''',
    charge = 0,                  # -1 = one extra electron
    spin = 0,                    # Number of unpared electrons (same as 2S) 
    output = 'output.out',       # Direct output from this molecule to a file 
    basis = '6-311g'             # Basis set used on the SCF process
    )


#Same molecule, but now with a different method, unrestricted DFT (PBE).
mf = scf.UKS(mol).newton()        # .newton() activates second-order self-consistent field (SOSCF)
mf = scf.addons.frac_occ(mf)      # Allow fractional orbital ocupation to help convergence

mf.xc = 'PBE'                     # Functional PBE. Any functional in XCLib can be used
mf.verbose = 4                    # Defines verbosity level. 0 = no output
mf.conv_tol = 1e-10               # Energy convergence criteria, in Eh
mf.max_cycle = 100                # Maximun allowed number of SCF cycles


mf.chkfile = 'chkpoint.dat'       # Checkpoint file to restart calculations
mf.init_guess = 'vsap'            # Strategy used for initial density guess
#mf.init_guess = 'chkpoint.dat'   # Loading old ckeckpoint file


#Runs the SCF
mf.kernel()

#Analyze the given SCF object: 
#    print orbital energies, occupancies; 
#    print orbital coefficients; 
#    Mulliken population analysis; 
#    Diople moment.
results = mf.analyze()

#This is how we can get forces from this calculation
force = mf.nuc_grad_method().kernel()

overwrite output file: output.out


Overwritten attributes  get_occ  of <class 'pyscf.soscf.newton_ah.newton.<locals>.SecondOrderUHF'>


Now, you can try to compute a particular system that you like. Maybe try to run something that has charges and different spin states. Maybe try to define a HF calculation? Or a small coupled cluster one? Check https://pyscf.org/quickstart.html or https://pyscf.org/user.html for more informations on how to do it.

Be aware that maybe you will need to play first with low  values of mf.conv_tol and mf.max_cycle, to be able to do it in the tutorial time.  

In [4]:
from pyscf import dft,gto,scf

#mol = gto.M(
#    atom = '''
#    XX  0.000   0.000   0.000''',
#    basis = '...'
#    )

#mf = ...


#mf.kernel()

Lets do one more step before egoing to ML. This is how we use PySCF to run DFT calculations with a customized functional, hard-coded.

`eval_xc` should be built to return the list [$𝜀_{XC}$,$v_{XC}$,$f_{XC}$,$k_{XC}$], i.e., $𝜀_{XC}$ and its first, second, and third derivatives respectively. In particular, $v_{XC}=(\frac{d𝜀_{XC}}{dρ},\frac{d𝜀_{XC}}{dγ},\frac{d𝜀_{XC}}{d\nabla^2ρ},\frac{d𝜀_{XC}}{dτ})$. This shape ensures the compatibility of our functional with the rest of PySCF.

To show how to customize a functional, we define a new functional that has the PBE correlation + PW86 exchange (for PW86 see https://doi.org/10.1103/physrevb.33.8800). Specifically, the PW86 exchange is here defined in terms of the reduced density gradients using the published parametrization by Perdew.

In [5]:
from pyscf import dft

def eval_xc(xc_code, rho, spin=0, relativity=0, deriv=1, omega=None, verbose=None):
    pi=3.1415926535897932384626433832795028841971
    third=1.0/3.0
    # A fictitious XC functional
    # xc_code = "LDA", "GGA", "meta-GGA"
    # SHAPE OF rho:      LDA - 1D array of shape (N) to store electron density, N being the number of integration grid points 
    #                    GGA - 2D array of shape (4,N) to store density and "density derivatives" for x,y,z components
    #                    meta-GGA - can be a (6,N) (with_lapl=True) array where last two rows are \nabla^2 rho and tau = 1/2(\nabla f)^2
    rho0, dx, dy, dz = rho[:4]
    gamma = (dx**2 + dy**2 + dz**2)
    
    kf=(3*pi**2)**third*(rho0+1e-6)**third
    s=(gamma)**0.5/(2*kf*(rho0+1e-6))
    a=0.0864
    b=14.0
    c=0.2
    m=1.0/15.0
    par=(1.0+(a/m)*s**2+b*s**4+c*s**6)
    Fx=par**m
    chainrule=m*par**(m-1.0) * ((2.0*a/m)*s+4*b*s**3+6*c*s**5)
    Fvrho   = chainrule * (4.0/3.0)*s/(rho0+1e-6)
    #Fvgamma = chainrule *    0.5   *s/(gamma+1e-6)

    # Getting original PBE Correlation and LDA Exchange
    pbe_c = dft.libxc.eval_xc(',pbe', rho, spin, relativity, deriv, verbose)   # 'pbe,pbe' is a explicit way of choosing that both exchange
    lda_x = dft.libxc.eval_xc('lda,',rho0, spin, relativity, deriv, verbose)   # and correlation will be done in the same way as 'PBE'.
                                                                               # 'pbe,' or ',pbe' can be used to get the X and C separetly.
                                                                               # The sintax is "exchange,correlation"
      
    

    # Mixing PBE and the fictitious functional
    exc = pbe_c[0] + Fx*lda_x[0]
    vrho = pbe_c[1][0] + Fvrho*lda_x[1][0]
    vgamma = pbe_c[1][1]
    vlapl = None
    vtau = None
    vxc = (vrho, vgamma, vlapl, vtau)
    fxc = None  # 2nd order functional derivative
    kxc = None  # 3rd order functional derivative

    return exc, vxc, fxc, kxc


mol = gto.M(
    atom = '''
    O  0.000   0.000   0.000
    H  0.000  -0.757   0.587
    H  0.000   0.757   0.587 ''',
    basis = '6-311g')

mf = dft.RKS(mol)
mf = mf.define_xc_(eval_xc, 'GGA', hyb=None)  # hyb is the amount of exact exchange to be used (HYBRID FUNCTIONAL)
                                              # 'GGA' is the xctype, and defines the shape of rho passed to eval_xc
mf.verbose = 0
mf.guess='huckel'
mf.kernel()


-63.629287665204714

---
## Crating a basic multilayer perceptron with TensorFlow


```   
          h - - - - h
       /  h - - - - h  \
     /    h - - - - h    \
   /      h - - - - h      \
i         h - - - - h        \
                               \
i         .         .            \
          .         .              o
i         .         .            /
                               /
i         h - - - - h        /
   \      h - - - - h      /
     \    h - - - - h    /
       \  h - - - - h  /
          h - - - - h
```
- 4 input neurons
- 100 neurons in hidden layer 1
- 100 neurons in hidden layer 2
- 1 output neuron

In [6]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import tensorflow as tf

def nn_model(x_input, W1, b1, W2, b2, W3, b3):
    x = tf.add(tf.matmul(tf.cast(x_input, tf.float32), W1), b1)      #
    x = tf.nn.relu(x)                                                #
    x = tf.add(tf.matmul(tf.cast(x, tf.float32), W2), b2)            # Bulding a multilayer perseptron using tf functions.
    x = tf.nn.relu(x)                                                #  - add and matmul correspond to basic matrix addition and multiplication
    x_output = tf.add(tf.matmul(x, W3), b3)                          #  - nn.relu is a build-in implementation of the activation function rELU(x)
    return x_output                                                  #


# weights connecting the input to the 1st hidden layer
W1 = tf.random.normal([4, 100], stddev=0.1)
b1 = tf.random.normal([100])
# weights connecting the hidden layers 
W2 = tf.random.normal([100, 100], stddev=0.1)
b2 = tf.random.normal([100])
# weights connecting the 2nd hidden layer to the output layer
W3 = tf.random.normal([100, 1], stddev=0.1)
b3 = tf.random.normal([1])

#Evaluating the ANN in random inputs, to showcase its usage. 
for i in range(10):
  input=tf.random.normal([1,4], stddev=100.0)
  print(input.numpy(),"\t",nn_model(input, W1, b1, W2, b2, W3, b3).numpy())

[[-131.07309   -51.745285  135.30573  -101.105156]] 	 [[28.83809]]
[[ 105.38672  -154.66153    55.68304   106.360176]] 	 [[13.542726]]
[[-158.54826    58.700634  -58.310425 -123.97181 ]] 	 [[15.2679]]
[[   5.223464   25.079119  -96.28534  -158.14421 ]] 	 [[21.506754]]
[[-61.575718   50.440155    5.0961313 100.081184 ]] 	 [[5.5114546]]
[[144.98987  -23.04728  -98.69745   15.573009]] 	 [[2.3169525]]
[[134.92712   18.2973    72.360146  19.26346 ]] 	 [[2.3254583]]
[[ 44.903133 144.73157   29.164146 -95.83941 ]] 	 [[8.413941]]
[[-163.37773    48.117264   96.65018    24.560034]] 	 [[19.502247]]
[[  58.209927  105.26288  -115.09255    96.38337 ]] 	 [[3.9280396]]





---

## Using a multilayer perceptron as functional


Here, we do a simple exemple of using $𝜀_{XC}=F_{XC}*𝜀_{XC}^{PBE}$, where $F_{XC}$ is the output of a multilayer perceptron. The inputs are the values of density (ρ) and the values of the gradients (γ=|∇ρ|), as the standard PBE.


```   
          h
       /  h  \
     /    h    \
   /      h      \
ρ         h        \
                     Fxc
γ         h        /
   \      h      /
     \    h    /
       \  h  /
          h
```



In [8]:
import tensorflow as tf

def nn_model(x_input, W1, b1, W2, b2):
    x = tf.add(tf.matmul(tf.cast(x_input, tf.float32), W1), b1)
    x = tf.nn.relu(x)
    logits = tf.add(tf.matmul(x, W2), b2)
    return logits

#We are setting the parameters randomly. You should not expect to see any good results from this... 
#Also, the convergence may take forever. So... Be free to play with setting max cycles 
W1 = tf.random.normal([2, 10], stddev=0.01)
b1 = tf.random.normal([10])
W2 = tf.random.normal([10, 1], stddev=0.01)
b2 = tf.random.normal([1])

from pyscf import gto
from pyscf import dft

def eval_xc(xc_code, rho, spin, relativity=0, deriv=1, omega=None, verbose=None):
    rho0, dx, dy, dz = rho[:4]
    gamma = (dx**2 + dy**2 + dz**2)**.5

    pbe_xc = dft.libxc.eval_xc('pbe,pbe', rho, spin, relativity, deriv, verbose)
    exc = pbe_xc[0]
    vrho = pbe_xc[1][0]
    vgamma = pbe_xc[1][1]
      
    Fxc = [nn_model([[rho0[i],gamma[i]]],W1, b1, W2, b2).numpy()[0][0] for i in range(len(rho0))]
    exc = Fxc * exc
    vrho = Fxc * vrho
    vgamma = Fxc * vgamma
    vlapl = None
    vtau = None
    vxc = (vrho, vgamma, vlapl, vtau)
    fxc = None 
    kxc = None 
    return exc, vxc, fxc, kxc

mol = gto.M(
    atom = '''
    O  0.000   0.000   0.000
    H  0.000  -0.757   0.587
    H  0.000   0.757   0.587 ''',
    basis = '6-311g')




mf = dft.RKS(mol)
mf = mf.define_xc_(eval_xc, 'GGA')
mf.verbose = 5
mf.conv_tol=1e-6
mf.max_cycle = 50
mf.kernel()




******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-06
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /content/tmp8e8mzcrn
max_memory 4000 MB (current use 575 MB)
XC library pyscf.dft.libxc version 6.1.0
    S. Lehtola, C. Steigemann, M. J.T. Oliveira, and M. A.L. Marques.,  SoftwareX 7, 1–5 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac.,  Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch.,  Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair.,  Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07
Set gradient conv threshold to 0.001
    CPU time for setting up grids      0.18 sec, wall time      0.17 sec
nelec by numeric integration = 9.99145148631681
    CPU time for vxc     16.85 sec, wall time     24.48 sec
E1 = -121.6917

-71.3287112611595

"Training" is nothing more than an optimization of the parameters of our ANN to minimize the loss function. To do so, we must construct the loss function inside a proper function that can be passed to your optimization library of choice. In this case, some intricate casting and shaping arrangements probably will be used.

I already did some "hard" work on training a very dummy functional on the Atomization Energy (AE) of water molecules only (a training set with a single molecule).  Here, we used $AE(H_2O)=E(H_2O)-E(H_2)-\tfrac{1}{2}E(O_2)$. I performed a few non-gradient-based optimizations using PySwarms (https://pyswarms.readthedocs.io/en/latest/).

Below, you can try to use some of those parameters (they are on the GitHub folder, maybe you will need to download and upload them to Colab by yourself...). To change between different sets, you can change "h=np.load('param6.npy')".

In [9]:
from pyscf import gto, scf, dft
import tensorflow as tf
import numpy as np


def rolling(parameters): #This is here just to "roll" the vector stored on the parameter files to the correct shapes used on the ANN.
	###  w1
	start=0
	end=start+2*10
	w1=parameters[start:end].reshape((2,10))
	###  b1
	start=end
	end=start+10
	b1=parameters[start:end].reshape((10,))
	###  w2
	start=end
	end=start+10*1
	w2=parameters[start:end].reshape((10,1))
	###  b2
	start=end
	end=start+1
	b2=parameters[start:end].reshape((1,))
	
	if end != len(parameters):
		print("Something is out of place... ")
		print("end=",end," but we expected it to be ", len(parameters))
		exit()
	
	return w1, b1, w2, b2

def eval_xc(xc_code, rho, spin=0, relativity=0, deriv=1, omega=None, verbose=None):
  rho0, dx, dy, dz = rho[:4]
  gamma = (dx**2 + dy**2 + dz**2)**.5
  G=[list(zip(rho0,gamma))]
   
  pbe_xc = dft.libxc.eval_xc('pbe,pbe', rho, spin, relativity, deriv, verbose)
  exc = pbe_xc[0]
  vrho = pbe_xc[1][0]
  vgamma = pbe_xc[1][1]
  
  # LOADING THE SAVED PARAMETER!!!
  h=np.load('param6.npy')
  W1, b1, W2, b2 = rolling(h)

  def nn_model(x_input, W1, b1, W2, b2):
    log2=0.6931471805599453094172321214581765680755001343602552541206800094
    x = tf.add(tf.matmul(tf.cast(x_input, tf.float32), W1), b1)
    x = tf.nn.softplus(x)/log2
    logits = tf.add(tf.matmul(x, W2), b2)
    y = tf.nn.softplus(logits)/log2
    return logits
  fxc = nn_model(G, W1, b1, W2, b2).numpy()
  exc = [fxc[0][i][0] * exc[i] for i in range(len(fxc[0]))]
  vrho = [fxc[0][i][0] * vrho[i] for i in range(len(fxc[0]))]
  vgamma = [fxc[0][i][0] * vgamma[i] for i in range(len(fxc[0]))]
  vlapl = None
  vtau = None
  vxc = (vrho[0], vgamma[0], vlapl, vtau)

  fxc = None  # 2nd order functional derivative
  kxc = None  # 3rd order functional derivative
  return exc, vxc, fxc, kxc



# Computing H2 for AE
mol = gto.M(
    atom = '''
    H        0.00000        0.00000   0.368583
    H        0.00000        0.00000  -0.368583''',
    basis = '6-311g')

# We obtain the B3LYP value (reference) and density
mf_mol = scf.RKS(mol)
mf_mol.xc = 'B3LYP'
mf_mol.verbose = 0
mf_mol.max_cycle = 200
mf_mol.init_guess = 'huckel'
e_tot_reff=mf_mol.kernel()
dmref = mf_mol.make_rdm1()

# Compute total energy with trained functional for B3LYP density
mf = dft.RKS(mol)
mf = mf.define_xc_(eval_xc, 'GGA')
mf.max_cycle = 0
mf.verbose = 0
e_tot_testf=mf.kernel(dm0=dmref)


# Compute PBE total energy for B3LYP density
mf = scf.RKS(mol)
mf.xc = 'PBE'
mf.verbose = 0
mf.max_cycle = 0
e_tot_PBE=mf.kernel(dm0=dmref)

print("H2:")
print("  B3LYP:   ",e_tot_reff)
e_tot_reff_H2=e_tot_reff*.5
print("  PBE:     ",e_tot_PBE)
e_tot_PBE_H2=e_tot_PBE*.5
print("  ANN-PBE: ",e_tot_testf)
e_tot_testf_H2=e_tot_testf*.5
print("-------------------------------------------------")

## Computing O2 for AE
mol = gto.M(
    atom =''' 
            O        0.00000000       0.00000000       0.62297800
            O        0.00000000       0.00000000      -0.62297800''',
    basis = '6-311g')
#B3LYP
mf_mol = scf.RKS(mol)
mf_mol.xc = 'B3LYP'
mf_mol.verbose = 0
mf_mol.max_cycle = 200
mf_mol.init_guess = 'huckel'
e_tot_reff=mf_mol.kernel()
dmref = mf_mol.make_rdm1()
#Trained functional
mf = dft.RKS(mol)
mf = mf.define_xc_(eval_xc, 'GGA')
mf.max_cycle = 0
mf.verbose = 0
e_tot_testf=mf.kernel(dm0=dmref)
#PBE
mf = scf.RKS(mol)
mf.xc = 'PBE'
mf.verbose = 0
mf.max_cycle = 0
e_tot_PBE=mf.kernel(dm0=dmref)

print("O2:")
print("  B3LYP:   ",e_tot_reff)
e_tot_reff_O2=e_tot_reff*.5
print("  PBE:     ",e_tot_PBE)
e_tot_PBE_O2=e_tot_PBE*.5
print("  ANN-PBE: ",e_tot_testf)
e_tot_testf_O2=e_tot_testf*.5
print("-------------------------------------------------")
print("=================================================")
print("-------------------------------------------------")




H2:
  B3LYP:    -1.169858401694336
  PBE:      -1.1630432931119747
  ANN-PBE:  -0.5595747577335771
-------------------------------------------------
O2:
  B3LYP:    -150.19872226867625
  PBE:      -150.12050078128482
  ANN-PBE:  -102.4605058537882
-------------------------------------------------
-------------------------------------------------


Check those energies... They make any sence? Why the ones we predict are so far from the B3LYP and PBE ones? Is it an issue?

In [10]:
# Error in H2O Atomization Energies (the one used for training)
mol = gto.M(
    atom = '''
        O  0.000   0.000   0.000
        H  0.000  -0.757   0.587
        H  0.000   0.757   0.587 ''',
    basis = '6-311g')

mf_mol = scf.RKS(mol)
mf_mol.xc = 'B3LYP'
mf_mol.verbose = 0
mf_mol.max_cycle = 200
mf_mol.init_guess = 'huckel'
e_tot_reff=mf_mol.kernel()
dmref = mf_mol.make_rdm1()

mf = dft.RKS(mol)
mf = mf.define_xc_(eval_xc, 'GGA')
mf.max_cycle = 0
mf.verbose = 0
e_tot_testf=mf.kernel(dm0=dmref)

mf = scf.RKS(mol)
mf.xc = 'PBE'
mf.verbose = 0
mf.max_cycle = 0
e_tot_PBE=mf.kernel(dm0=dmref)

print("-------------------------------------------------")
deltaE=abs((e_tot_reff-2.0*e_tot_reff_H2-e_tot_reff_O2)-(e_tot_testf-2.0*e_tot_testf_H2-e_tot_testf_O2))
deltaEPBE=abs((e_tot_reff-2.0*e_tot_reff_H2-e_tot_reff_O2)-(e_tot_PBE-2.0*e_tot_PBE_H2-e_tot_PBE_O2))
print("H2O (loss function):")
print(" Err.ANN:",deltaE)
print(" Err.PBE:",deltaEPBE)
print("-------------------------------------------------")
print("-------------------------------------------------")


# Error on AE for H2O2
mol = gto.M(
    atom = '''
    O        0.00000000       0.73405800      -0.05275000
    O        0.00000000      -0.73405800      -0.05275000
    H        0.83954700       0.88075200       0.42200100
    H       -0.83954700      -0.88075200       0.42200100''',
    basis = '6-311g')

mf_mol = scf.RKS(mol)
mf_mol.xc = 'B3LYP'
mf_mol.verbose = 0
mf_mol.max_cycle = 200
mf_mol.init_guess = 'huckel'
e_tot_reff=mf_mol.kernel()
dmref = mf_mol.make_rdm1()

mf = dft.RKS(mol)
mf = mf.define_xc_(eval_xc, 'GGA')
mf.max_cycle = 0
mf.verbose = 0
e_tot_testf=mf.kernel(dm0=dmref)

mf = scf.RKS(mol)
mf.xc = 'PBE'
mf.verbose = 0
mf.max_cycle = 0
e_tot_PBE=mf.kernel(dm0=dmref)

print("-------------------------------------------------")
deltaE=abs((e_tot_reff-2.0*e_tot_reff_H2-2.0*e_tot_reff_O2)-(e_tot_testf-2.0*e_tot_testf_H2-2.0*e_tot_testf_O2))
deltaEPBE=abs((e_tot_reff-2.0*e_tot_reff_H2-2.0*e_tot_reff_O2)-(e_tot_PBE-2.0*e_tot_PBE_H2-2.0*e_tot_PBE_O2))
print("H2O2:")
print(" Err.ANN:",deltaE)
print(" Err.PBE:",deltaEPBE)
print("-------------------------------------------------")



# Error on AE for ozone (O3)
mol = gto.M(
    atom = '''
    O        0.00000        1.10381  -0.228542
    O        0.00000        0.00000   0.457084
    O        0.00000       -1.10381  -0.228542''',
    basis = '6-311g')

mf_mol = scf.RKS(mol)
mf_mol.xc = 'B3LYP'
mf_mol.verbose = 0
mf_mol.max_cycle = 200
mf_mol.init_guess = 'huckel'
e_tot_reff=mf_mol.kernel()
dmref = mf_mol.make_rdm1()

mf = dft.RKS(mol)
mf = mf.define_xc_(eval_xc, 'GGA')
mf.max_cycle = 0
mf.verbose = 0
e_tot_testf=mf.kernel(dm0=dmref)

#PBE 
mf = scf.RKS(mol)
mf.xc = 'PBE'
mf.verbose = 0
mf.max_cycle = 0
e_tot_PBE=mf.kernel(dm0=dmref)

print("-------------------------------------------------")
deltaE=abs((e_tot_reff-3.0*e_tot_reff_O2)-(e_tot_testf-3.0*e_tot_testf_O2))
deltaEPBE=abs((e_tot_reff-3.0*e_tot_reff_O2)-(e_tot_PBE-3.0*e_tot_PBE_O2))
print("O3:")
print(" Err.ANN:",deltaE)
print(" Err.PBE:",deltaEPBE)
print("-------------------------------------------------")




-------------------------------------------------
H2O (loss function):
 Err.ANN: 0.002064871581936245
 Err.PBE: 0.004229887915855102
-------------------------------------------------
-------------------------------------------------
-------------------------------------------------
H2O2:
 Err.ANN: 6.461784484201871e-05
 Err.PBE: 0.002372846581039312
-------------------------------------------------
-------------------------------------------------
O3:
 Err.ANN: 0.031016918380743164
 Err.PBE: 0.021543374542233096
-------------------------------------------------


Which molecules are predicted in a correct way and which are not? In which ones we improved over PBE?



---

#Some things that you can try for yourself:



1. Instead of correcting PBE, why not a Meta-GGA? Think on how the code needs to change.
2. Check those parameters for self-consistent calculations. (SPOILER: They are bad. Maybe one can optimize everything using self-consistent energies... Maybe it can be you, in case you do the next suggestions.)
3.   Implement the atomization energy of a small set of molecules as a function of the parameters (as a callable object). Maybe the structures can come from ase.collections (https://wiki.fysik.dtu.dk/ase/ase/collections.html#ase.collections.g2)
4.   Send this loss function to a non-gradient-based optimization library and create your own functional. 
5.   Automate the validation for a set of molecules that extend on your training. 



In [11]:
#...